# примитивный вариант решения задачи MKB

автор: Александр Дьяконов (https://dyakonov.org/ag/)

цель: для оценки студентов своего курса (нужно за неделю побить этот бенчмарк)

* решение записано за 20 минут
* практически нет генерации признаков
* все категории кодируются по мощности
* пропуски -> -1
* одна модель - lgb

результат в лидерборе 0.8889 (на момент посылки ~15 место из 100)

In [1]:
import numpy as np
import pandas as pd

In [2]:
data_train = pd.read_csv('train_dataset_hackathon_mkb.csv', encoding='cp1251', delimiter=';')
data_test = pd.read_csv('test_dataset_hackathon_mkb.csv', encoding='cp1251', delimiter=';')
print (data_train.shape, data_test.shape)

(17891, 124) (7330, 123)


In [3]:
def makeX(data):
    # предобработка данных
    data['CITIZENSHIP_NAME'] = data['CITIZENSHIP_NAME'].fillna(-1).map({-1: -1, 'Российская Федерация': 4, 'Таджикистан': 3, 'Казахстан': 2, 'Армения': 1})
    data['SEX_NAME'] = data['SEX_NAME'].fillna(0).map({0: 0, 'мужской': 1, 'женский': -1})
    group_names = ['OKFS_GROUP', 'OKOPF_GROUP', 'OKOGU_GROUP'] + ['WORKERSRANGE', 'OKVED_CODE']
    date_names = ['SIGN_DATE', 'DATEFIRSTREG', 'TAXREG_REGDATE', 'TAXREGPAY_REGDATE', 'BIRTHDATE']
    for name in group_names + date_names + ['id_client']:
        data[name] = data[name].fillna(-1)
        tmp = data[name].value_counts()
        tmp = tmp + 0.1 * np.random.randn(len(tmp))
        data[name] = data[name].map(tmp)
    data.fillna(-1, inplace=True)
    return data

In [4]:
data_train = makeX(data_train) # обрабатываем обучение
data_test = makeX(data_test) # обрабатываем тест

In [5]:
y = data_train.pop('TARGET').values # целевые значения
data_test = data_test[data_train.columns] # на всякий случай - вдруг, перемешаны столбцы

In [6]:
import lightgbm as lgb

model = lgb.LGBMClassifier(num_leaves=31,
                           learning_rate=0.05,
                           n_estimators=200)

In [7]:
model.fit(data_train, y)

a = model.predict_proba(data_test)[:, 1] # получаем ответ

In [8]:
df = pd.DataFrame({'id_contract': data_test.id_contract.values, 'TARGET': a})
df.to_csv('ans1.csv', sep=';', index=False) # сохраняем ответ